# Convolutional Neural Network (CNN)

<img src="https://i.imgur.com/H8yBJsy.jpg" alt="drawing" width="800px"/>

* Import
* Set The Device
* Dataset - MNIST
* Model: CNN
* Loss Function & Optimizer
* Training Model
* Testing Model
* Save & Load Model

## Import

In [15]:
import torch

# for Dataset
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader

# for Training Model
import torch.nn as nn
import torch.optim as opt

## Set The Device

In [16]:
torch.cuda.get_device_name()

'Tesla T4'

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Dataset - MNIST

In [18]:
train_dataset = MNIST(root = './data', train=True, download=True, transform=transforms.ToTensor())
test_dataset= MNIST(root = './data', train=False, download=False, transform=transforms.ToTensor())

## DataLoader

In [19]:
# hyperparameter 
train_batch_size = 64
test_batch_szie = 64

# train dataloader
train_loader = DataLoader(
    dataset=train_dataset, 
    batch_size=train_batch_size, 
    shuffle=True
    )

# test dataloader
test_loader = DataLoader(
    dataset=test_dataset, 
    batch_size=test_batch_szie, 
    shuffle=False
)

## Model

In [20]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(stride=2, kernel_size=2)
        )
        self.dense = nn.Sequential(
            nn.Linear(in_features=14*14*128, out_features=1024),
            nn.ReLU(),
            nn.Linear(1024, 10)
        )

    def forward(self, x):
        output = self.conv_layers(x) # 
        output = output.view(-1, 14*14*128)
        output = self.dense(output)
        return output

In [21]:
model = CNN()
model.to(device)

CNN(
  (conv_layers): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (dense): Sequential(
    (0): Linear(in_features=25088, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=10, bias=True)
  )
)

## Loss Function & Optimizer

In [22]:
# hypyerperameter
learning_rate = 0.0001

loss_func = nn.CrossEntropyLoss()
optimizer = opt.Adam(model.parameters(), lr=learning_rate)

## Training Model


In [23]:
# hyperparameter 
num_epochs = 5

for epoch in range(num_epochs):
    for idx, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        if (idx+1)%100 == 0:
            print("Epoch: %d, Batch: %d, Loss: %.4f" %(epoch+1, idx+1, loss.data))

Epoch: 1, Batch: 100, Loss: 0.4460
Epoch: 1, Batch: 200, Loss: 0.2841
Epoch: 1, Batch: 300, Loss: 0.0730
Epoch: 1, Batch: 400, Loss: 0.1334
Epoch: 1, Batch: 500, Loss: 0.1958
Epoch: 1, Batch: 600, Loss: 0.0555
Epoch: 1, Batch: 700, Loss: 0.0945
Epoch: 1, Batch: 800, Loss: 0.1423
Epoch: 1, Batch: 900, Loss: 0.0608
Epoch: 2, Batch: 100, Loss: 0.0468
Epoch: 2, Batch: 200, Loss: 0.0529
Epoch: 2, Batch: 300, Loss: 0.0268
Epoch: 2, Batch: 400, Loss: 0.0610
Epoch: 2, Batch: 500, Loss: 0.0084
Epoch: 2, Batch: 600, Loss: 0.0562
Epoch: 2, Batch: 700, Loss: 0.0137
Epoch: 2, Batch: 800, Loss: 0.0108
Epoch: 2, Batch: 900, Loss: 0.0232
Epoch: 3, Batch: 100, Loss: 0.0093
Epoch: 3, Batch: 200, Loss: 0.0053
Epoch: 3, Batch: 300, Loss: 0.0551
Epoch: 3, Batch: 400, Loss: 0.0632
Epoch: 3, Batch: 500, Loss: 0.0424
Epoch: 3, Batch: 600, Loss: 0.0069
Epoch: 3, Batch: 700, Loss: 0.0502
Epoch: 3, Batch: 800, Loss: 0.0325
Epoch: 3, Batch: 900, Loss: 0.0327
Epoch: 4, Batch: 100, Loss: 0.0173
Epoch: 4, Batch: 200

## Testing Model

In [25]:
correct = 0
total = 0
for images, labels in test_loader:
  images = images.to(device)
  outputs = model(images)

  _, pred = torch.max(outputs.data, 1)
  
  correct += (pred == labels.to(device)).sum()
  total += labels.size(0)

print('Accuracy:%.3f%%' %(100.0 * float(correct)/float(total)))

Accuracy:98.680%


## Save & Load Model

* Method 1: save model's weight
* Method 2: save entire model

In [26]:
# method 1: save model weight
torch.save(model.state_dict(), 'model.pkl')

# load weight
model = CNN()
model.load_state_dict(torch.load('model.pkl'))

<All keys matched successfully>

In [27]:
torch.save(model, 'model3.pkl')